In [1]:
import h5py,pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Aquisition des données pour utilisation dans Jupyter (local)

In [2]:
def dataread():
    X_file=h5py.File("C:/Users/AntoineLespinasse/Desktop/STATS/Projet/data/X_train_new.h5",'r') # X est ici au format "HDF5 dataset"
    X=np.array(X_file['features']) # X est ici au format "ndarray"
    
    X_final_file=h5py.File("C:/Users/AntoineLespinasse/Desktop/STATS/Projet/data/X_test_new.h5",'r') # X_final est ici au format "HDF5 dataset"
    X_final=np.array(X_final_file['features']) # X_final est ici au "ndarray"

    y=pd.read_csv("C:/Users/AntoineLespinasse/Desktop/STATS/Projet/data/y_train_AvCsavx.csv") # y est ici au format "pandas DataFrame"
    y=np.array(y['label']) # y est ici au format "ndarray"

    return X, y, X_final

Aquisition des données pour utilisation dans Collab (cloud)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def dataread():
    X_file=h5py.File("drive/MyDrive/X_train_new.h5",'r') # X est ici au format "HDF5 dataset"
    X=np.array(X_file['features']) # X est ici au format "ndarray"
    
    X_final_file=h5py.File("drive/MyDrive/X_test_new.h5",'r') # X_final est ici au format "HDF5 dataset"
    X_final=np.array(X_final_file['features']) # X_final est ici au "ndarray"

    y=pd.read_csv("drive/MyDrive/y_train_AvCsavx.csv") # y est ici au format "pandas DataFrame"
    y=np.array(y['label']) # y est ici au format "ndarray"

    return X, y, X_final

In [4]:
def datatreat_1(X0, y0):
    X=[]
    y=[]
    for i in range(np.shape(X0)[0]):
        for j in range(np.shape(X0)[1]):
            X.append(X0[i,j,:,:])
            y.append(y0[i])

    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.6, shuffle=True)

    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [5]:
def cnn_1():
  model = keras.Sequential(
    [
      layers.Conv2D(100, (3,3), padding='same', activation='relu', input_shape=(7,500,1)),
      layers.MaxPool2D((2,2), padding='same'),
      layers.Conv2D(100, (3,3), padding='same', activation='relu'),
      layers.MaxPool2D((2,2), padding='same'),
      layers.Conv2D(300, (2,3), padding='same', activation='relu'),
      layers.MaxPool2D((2,2), padding='same'),
      layers.Conv2D(400, (1,7), padding='same', activation='relu'),
      layers.MaxPool2D((1,2), padding='same'),
      layers.Conv2D(100, (1,3), padding='same', activation='relu'),
      layers.Conv2D(100, (1,3), padding='same', activation='relu'),
      layers.Flatten(),
      layers.Dense(50, activation='relu'),
      layers.Dense(1, activation='sigmoid')
    ]
  )

  model.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy']
  )

  return model

In [6]:
X, y, X_final = dataread()

X_train, X_test, y_train, y_test = datatreat_1(X, y)

X_train = X_train.reshape(X_train.shape[0], 7, 500, 1)
X_test = X_test.reshape(X_test.shape[0], 7, 500, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(22704, 7, 500, 1)
(15136, 7, 500, 1)
(22704,)
(15136,)


In [7]:
benchmark = cnn_1()

print(benchmark.summary())

benchmark.fit(X_train, y_train, epochs=6, batch_size=64)

print("Evaluate on test data")
results = benchmark.evaluate(X_test, y_test)
print("test loss, test acc:", results)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 500, 100)       1000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 250, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 4, 250, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 125, 100)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 125, 300)       180300    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 63, 300)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 63, 400)        8